In [83]:
from stravalib.client import Client
import time
import pandas as pd

import seaborn as sns
sns.set(font_scale=1.5)
sns.set_style("whitegrid")

import matplotlib.pyplot as plt
%matplotlib inline

## Load Data From Strava

## Web access post reuquest Demo (https://developers.strava.com/docs/getting-started/#oauth)
curl -X POST https://www.strava.com/oauth/token \
    -F client_id=40761 \
    -F client_secret=72638d8c256371fea5e3a90819267d838a7de254 \
    -F code=005352d0601908e4ae3f8748141d14583197feb8 \
    -F grant_type=authorization_code

In [58]:
client = Client()
authorize_url = client.authorization_url(client_id=1234, redirect_uri='http://localhost:8282/authorized')
# Have the user click the authorization URL, a 'code' param will be added to the redirect_uri
# .....

# Extract the code from your webapp response

code = '57e0fff2b106d7dbe92246b35936cdcec9b28b1a'
client_id = 40761
client_secret = '72638d8c256371fea5e3a90819267d838a7de254'


# code = request.get('code') # or whatever your framework does
token_response = client.exchange_code_for_token(client_id=client_id, client_secret=client_secret, code=code)
access_token = token_response['access_token']
refresh_token = token_response['refresh_token']
expires_at = token_response['expires_at']

# Now store that short-lived access token somewhere (a database?)
client.access_token = access_token
# You must also store the refresh token to be used later on to obtain another valid access token 
# in case the current is already expired
client.refresh_token = refresh_token

# An access_token is only valid for 6 hours, store expires_at somewhere and
# check it before making an API call.
client.token_expires_at = expires_at
print(f'expires at: {expires_at}')
 
athlete = client.get_athlete()
print("For {id}, I now have an access token {token}".format(id=athlete.id, token=access_token))

if time.time() > client.token_expires_at:
    refresh_response = client.refresh_access_token(client_id=1234, client_secret='asdf1234',
        refresh_token=client.refresh_token)
    access_token = refresh_response['access_token']
    refresh_token = refresh_response['refresh_token']
    expires_at = refresh_response['expires_at']

expires at: 1573740692
For 23066318, I now have an access token c0cb83df4e253400baf3a3ecf5beb9c7453e5118


## Get some stats on current athlete (me)

In [59]:
client =  Client(access_token=access_token)

In [60]:
print("Hello, {}".format(athlete.firstname))

Hello, Nate


In [61]:
for activity in client.get_activities(after = "2010-01-01T00:00:00Z",  limit=5):
    print("{0.name} {0.moving_time}".format(activity))

Keystone Day 4 3:24:42
Eldora XC skiing 1:58:05
Muddy run w/ Zach 0:56:00
New Albany 0:23:24
Gahanna / New Albany 0:31:45


In [65]:
activity_raw = client.get_activities(limit=1000)

In [66]:
activities = list(activity_raw)

In [71]:
print(len(activities))

152


## Data Test

In [80]:
d = {
    'high' : [a.elev_high for a in activities],
    'low' :  [a.elev_low for a in activities]
}
elevation_range = pd.DataFrame.from_dict(d).dropna()
elevation_range.head()

,high,low
1,1775.0,1606.8
2,1657.2,1632.0
3,295.2,286.6
4,312.2,284.2
5,314.8,286.8


### Drop bad data (why is it bad?)

In [99]:
elevation_range = elevation_range[elevation_range.high -  elevation_range.low < 5000]

In [100]:
#  Note: Added the "if" condition because there seems to be a bad day
y_plot_data = [(y_min, y_max) for y_min, y_max  in zip(elevation_range.low, elevation_range.high)]

In [ ]:
f, axarr = plt.subplots(figsize=(20, 10), dpi=100)
sns.boxplot(x=elevation_range.index, y=y_plot_data)
plt.title('Elevation Range Over Strava Activity #')
plt.show()